In [1]:
from src.value import Value
import src.function as F
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = Value(X_train, "x_train", requires_grad=False)
X_test = Value(X_test, "x_test", requires_grad=False)
y_train = Value(y_train, "y_train", requires_grad=False)
y_test = Value(y_test, "y_test", requires_grad=False)

In [3]:
w = Value(np.random.randn(X.shape[1]) * 10**(-2), "w", requires_grad = True)
b = Value([0.0], "b", requires_grad = True)

In [4]:
lr = 0.01

matmul1 = F._FunctionFactory().get_new_function_of_type(F._Matmul)
add1 = F._FunctionFactory().get_new_function_of_type(F._Add)
loss = F._FunctionFactory().get_new_function_of_type(F._BCELossWithLogits)
    
def fit(X, num_epoch):     
    for i in range(num_epoch):
        
        y = add1.forward(matmul1.forward(X, w), b)
        l = loss.forward(y, y_train)
        loss.backward()
        w.value -= lr * w.grad
        b.value -= lr * b.grad
        l.zero_grad()   
        
        # if i % 10:
        print(f"Epoch {i+1}/{num_epoch}: Train Loss: {l.value}")

def predict(X):
    y = add1.forward(matmul1.forward(X, w), b)
    y_pred = F.sigmoid(y).value > 0.5
    
    return y_pred

Creating Function of type <class 'src.function._Matmul'> with name matmul_0
Creating Function of type <class 'src.function._Add'> with name add_0
Creating Function of type <class 'src.function._BCELossWithLogits'> with name bce_loss_logit_0


In [5]:
fit(X_train, 100)    

Epoch 1/100: Train Loss: 0.6984120886333116
Epoch 2/100: Train Loss: 0.37658408260145604
Epoch 3/100: Train Loss: 0.20896615325387793
Epoch 4/100: Train Loss: 0.13335055435116958
Epoch 5/100: Train Loss: 0.11048728246836521
Epoch 6/100: Train Loss: 0.09946329512497668
Epoch 7/100: Train Loss: 0.09283392046038026
Epoch 8/100: Train Loss: 0.08852153206254723
Epoch 9/100: Train Loss: 0.08531219286417871
Epoch 10/100: Train Loss: 0.08271671865516182
Epoch 11/100: Train Loss: 0.0805084082834039
Epoch 12/100: Train Loss: 0.07856670482119812
Epoch 13/100: Train Loss: 0.07682251194938303
Epoch 14/100: Train Loss: 0.07523384311698357
Epoch 15/100: Train Loss: 0.07377355493949667
Epoch 16/100: Train Loss: 0.07242294737104696
Epoch 17/100: Train Loss: 0.07116836376817401
Epoch 18/100: Train Loss: 0.06999932917808133
Epoch 19/100: Train Loss: 0.0689074840993852
Epoch 20/100: Train Loss: 0.06788594105830838
Epoch 21/100: Train Loss: 0.06692887594595237
Epoch 22/100: Train Loss: 0.06603125659719221


In [6]:
y_train_pred = predict(X_train)
y_test_pred = predict(X_test)

Creating Function of type <class 'src.function._Sigmoid'> with name sigmoid_0
Creating Function of type <class 'src.function._Sigmoid'> with name sigmoid_1


In [7]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve, matthews_corrcoef

def print_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)    

    print(f"Confusion Matrix")
    print(f"{conf_matrix}\n")
    print(f"Accuracy: {(tp + tn)/(tp + tn + fn + fp)}")
    print(f"Precision: {tp / (tp + fp)}")
    print(f"Recall: {tp / (tp + fn)}")
    print(f"AUC score: {auc(fpr, tpr)}")
    print(f"MCC score: {mcc}")
    print(f"F1-Score: {(tp)/(tp+(fp+fn)/2)}\n")
    
print_metrics(y_train.value, y_train_pred)
print("")
print_metrics(y_test.value, y_test_pred)

ValueError: Expected array-like (array or non-string sequence), got Value(data=[1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 0 0 0
 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1
 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0
 1 1 1 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 1 0 1 0
 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1
 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0
 0 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1
 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 1 0
 0 1 1 0 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0 1 1 1 0
 0 0 1 0 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0
 0 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1], shape=(426,), name=y_train, requires_grad=False), dtype